# 含并行连结的网络（GoogLeNet）\n\n:label:`sec_googlenet`

在2014年的ImageNet图像识别挑战赛中，一个名叫*GoogLeNet* :cite:`Szegedy.Liu.Jia.ea.2015`的网络架构大放异彩。\nGoogLeNet吸收了NiN中串联网络的思想，并在此基础上做了改进。\n这篇论文的一个重点是解决了什么样大小的卷积核最合适的问题。\n毕竟，以前流行的网络使用小到$1 \\times 1$，大到$11 \\times 11$的卷积核。\n本文的一个观点是，有时使用不同大小的卷积核组合是有利的。\n本节将介绍一个稍微简化的GoogLeNet版本：我们省略了一些为稳定训练而添加的特殊特性，现在有了更好的训练方法，这些特性不是必���的。

## (**Inception块**)

在GoogLeNet中，基本的卷积块被称为*Inception块*（Inception block）。这很可能得名于电影《盗梦空间》（Inception），因为电影中的一句话“我们需要走得更深”（“We need to go deeper”）。

![Inception块的架构。](../img/inception.svg)\n:label:`fig_inception`

如 :numref:`fig_inception`所示，Inception块由四条并行路径组成。\n前三条路径使用窗口大小为$1\\times 1$、$3\\times 3$和$5\\times 5$的卷积层，从不同空间大小中提取信息。\n中间的两条路径在输入上执行$1\\times 1$卷积，以减少通道数，从而降低模型的��杂性。\n第四条路径使用$3\\times 3$最大汇聚层，然后使用$1\\times 1$卷积层来改变通道数。\n这四条路径都使用合适的填充来使输入与输出的高和宽一致，最后我们将每条线路的输出在通道维度上连结，并构成Inception块的输出。在Inception块中，通常调整的超参数是每层输出通道数。

In [ ]:
import torch\nfrom torch import nn\nfrom torch.nn import functional as F\nfrom d2l import torch as d2l

In [ ]:
class Inception(nn.Module):\n    # c1--c4是每条路径的输出通道数\n    def __init__(self, in_channels, c1, c2, c3, c4, **kwargs):\n        super(Inception, self).__init__(**kwargs)\n        # 线路1，单1x1卷积层\n        self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)\n        # 线路2，1x1卷积层后接3x3卷积层\n        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)\n        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)\n        # 线路3，1x1卷积层后接5x5卷积层\n        self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)\n        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)\n        # 线路4，3x3最大汇聚层后接1x1卷积层\n        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)\n        self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)\n\n    def forward(self, x):\n        p1 = F.relu(self.p1_1(x))\n        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))\n        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))\n        p4 = F.relu(self.p4_2(self.p4_1(x)))\n        # 在通道维度上连结输出\n        return torch.cat((p1, p2, p3, p4), dim=1)

## [**GoogLeNet模型**]

现在，我们逐一实现GoogLeNet的每个模块。第一个模块使用64个通道、$7\\times 7$卷积层。

In [ ]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),\n                   nn.ReLU(),\n                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

第二个模块使用两个卷积层：第一个卷积层是64个通道、$1\\times 1$卷积层；第二个卷积层使用将通道数量增加三倍的$3\\times 3$卷积层。

In [ ]:
b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),\n                   nn.ReLU(),\n                   nn.Conv2d(64, 192, kernel_size=3, padding=1),\n                   nn.ReLU(),\n                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

第三个模块串联两个完整的Inception块。

In [ ]:
b3 = nn.Sequential(Inception(192, 64, (96, 128), (16, 32), 32),\n                   Inception(256, 128, (128, 192), (32, 96), 64),\n                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

第四模块更加复杂，它串联了5个Inception块

In [ ]:
b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64),\n                   Inception(512, 160, (112, 224), (24, 64), 64),\n                   Inception(512, 128, (128, 256), (24, 64), 64),\n                   Inception(512, 112, (144, 288), (32, 64), 64),\n                   Inception(528, 256, (160, 320), (32, 128), 128),\n                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

第五模块包含两个Inception块。

In [ ]:
b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),\n                   Inception(832, 384, (192, 384), (48, 128), 128),\n                   nn.AdaptiveAvgPool2d((1,1)),\n                   nn.Flatten())

In [ ]:
net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(1024, 10))

下面演��各个模块输出的形状变化。

In [ ]:
X = torch.rand(size=(1, 1, 96, 96))\nfor layer in net:\n    X = layer(X)\n    print(layer.__class__.__name__,'output shape:\t', X.shape)

## [**训练模型**]

In [ ]:
lr, num_epochs, batch_size = 0.1, 10, 128\ntrain_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)\nd2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())